# LIAR DETECTION GROUP PROJECT

Run the cell below to import packages.

In [2]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
from functools import reduce

#import unittest
from IPython.display import display, HTML
#from sklearn.utils import shuffle
# NLTK for NLP utils and corpora
#import nltk
from collections import defaultdict
# NumPy and TensorFlow
import numpy as np
import pandas as pd
#import tensorflow as tf
import pickle
import dill
# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz
import timeit  #For timing

#from project_files import pdio # for saving and loading dataframes

### Load data
Loading data from Pre-Processing step Part 1

In [3]:
#Read pkl file from part 1
all_data = pd.read_pickle('parsed_data/df_alldata1.pkl')
all_data.info(memory_usage='deep', verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
title      44898 non-null object
text       44898 non-null object
subject    44898 non-null object
date       44898 non-null object
target     44898 non-null object
dtypes: object(5)
memory usage: 151.9 MB


### Process character n-grams
Create character n-grams for each of the news entries

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

def get_char_ngram_vectorizer(text_list, ngram_len):
    ngrams = collections.Counter()
    for item in text_list:
        # remove extra white space
        text_tokens = item.split()
        new_item = ' '.join(text_tokens)

        # generate character ngrams
        char_ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(ngram_len, ngram_len))
        char_ngram_vectorizer.fit_transform(text_list)
    return char_ngram_vectorizer

true_titles = list(all_data.loc[all_data['target']=='1']['title'][:5])
print(true_titles)

my_ngram_vec = get_char_ngram_vectorizer(true_titles, 5)
my_ngram_vec.get_feature_names()

# len(my_ngrams)

["North Korea's Kim Jong Un fetes nuclear scientists, holds celebration bash", 'White House developing comprehensive biosecurity strategy: official', 'Republican Senator Alexander to consult on bipartisan healthcare plan', 'Zimbabwe\'s army seizes power, Mugabe confined but "safe"', 'U.S. senator urges DOJ to reject any White House push in merger probes']


[' "saf',
 ' alex',
 ' any ',
 ' army',
 ' bash',
 ' bios',
 ' bipa',
 ' but ',
 ' cele',
 ' comp',
 ' conf',
 ' cons',
 ' deve',
 ' doj ',
 ' fete',
 ' heal',
 ' hold',
 ' hous',
 ' in ',
 ' jong',
 ' kim ',
 ' kore',
 ' merg',
 ' muga',
 ' nort',
 ' nucl',
 ' offi',
 ' on ',
 ' plan',
 ' powe',
 ' prob',
 ' push',
 ' reje',
 ' repu',
 ' scie',
 ' seiz',
 ' sena',
 ' stra',
 ' to ',
 ' u.s.',
 ' un ',
 ' urge',
 ' whit',
 ' zimb',
 '"safe',
 "abwe'",
 'afe" ',
 'alexa',
 'althc',
 'ander',
 'army ',
 'artis',
 'ategy',
 'ation',
 'ator ',
 'babwe',
 'bash ',
 'biose',
 'bipar',
 'blica',
 'brati',
 "bwe's",
 'care ',
 'celeb',
 'cial ',
 'cient',
 'clear',
 'compr',
 'confi',
 'consu',
 'curit',
 'devel',
 "ea's ",
 'ealth',
 'ebrat',
 'ecuri',
 'egy: ',
 'ehens',
 'eizes',
 'eject',
 'elebr',
 'elopi',
 'enato',
 'ensiv',
 'entis',
 'epubl',
 'erger',
 'etes ',
 'evelo',
 'exand',
 'fetes',
 'ffici',
 'ficia',
 'fined',
 'gabe ',
 'hcare',
 'healt',
 'hensi',
 'hite ',
 'holds',
 'ho

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

def get_char_ngram_vectorizer(text_list, ngram_len):
    ngrams = collections.Counter()
    for item in text_list:
        # remove extra white space
        text_tokens = item.split()
        new_item = ' '.join(text_tokens)

        # generate character ngrams
        char_ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(ngram_len, ngram_len))
        char_ngram_vectorizer.fit_transform(text_list)
    return char_ngram_vectorizer

true_titles = list(all_data.loc[all_data['target']=='1']['title'][:5])
print(true_titles)

my_ngram_vec = get_char_ngram_vectorizer(true_titles, 5)
my_ngram_vec.get_feature_names()

# len(my_ngrams)

["North Korea's Kim Jong Un fetes nuclear scientists, holds celebration bash", 'White House developing comprehensive biosecurity strategy: official', 'Republican Senator Alexander to consult on bipartisan healthcare plan', 'Zimbabwe\'s army seizes power, Mugabe confined but "safe"', 'U.S. senator urges DOJ to reject any White House push in merger probes']


[' "saf',
 ' alex',
 ' any ',
 ' army',
 ' bash',
 ' bios',
 ' bipa',
 ' but ',
 ' cele',
 ' comp',
 ' conf',
 ' cons',
 ' deve',
 ' doj ',
 ' fete',
 ' heal',
 ' hold',
 ' hous',
 ' in ',
 ' jong',
 ' kim ',
 ' kore',
 ' merg',
 ' muga',
 ' nort',
 ' nucl',
 ' offi',
 ' on ',
 ' plan',
 ' powe',
 ' prob',
 ' push',
 ' reje',
 ' repu',
 ' scie',
 ' seiz',
 ' sena',
 ' stra',
 ' to ',
 ' u.s.',
 ' un ',
 ' urge',
 ' whit',
 ' zimb',
 '"safe',
 "abwe'",
 'afe" ',
 'alexa',
 'althc',
 'ander',
 'army ',
 'artis',
 'ategy',
 'ation',
 'ator ',
 'babwe',
 'bash ',
 'biose',
 'bipar',
 'blica',
 'brati',
 "bwe's",
 'care ',
 'celeb',
 'cial ',
 'cient',
 'clear',
 'compr',
 'confi',
 'consu',
 'curit',
 'devel',
 "ea's ",
 'ealth',
 'ebrat',
 'ecuri',
 'egy: ',
 'ehens',
 'eizes',
 'eject',
 'elebr',
 'elopi',
 'enato',
 'ensiv',
 'entis',
 'epubl',
 'erger',
 'etes ',
 'evelo',
 'exand',
 'fetes',
 'ffici',
 'ficia',
 'fined',
 'gabe ',
 'hcare',
 'healt',
 'hensi',
 'hite ',
 'holds',
 'ho

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

def get_char_ngram_vectorizer(text_list, ngram_len):
    ngrams = collections.Counter()
    for item in text_list:
        # remove extra white space
        text_tokens = item.split()
        new_item = ' '.join(text_tokens)

        # generate character ngrams
        char_ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(ngram_len, ngram_len))
        char_ngram_vectorizer.fit_transform(text_list)
    return char_ngram_vectorizer

true_titles = list(all_data.loc[all_data['target']=='1']['title'][:5])
print(true_titles)

my_ngram_vec = get_char_ngram_vectorizer(true_titles, 5)
my_ngram_vec.get_feature_names()

# len(my_ngrams)

["North Korea's Kim Jong Un fetes nuclear scientists, holds celebration bash", 'White House developing comprehensive biosecurity strategy: official', 'Republican Senator Alexander to consult on bipartisan healthcare plan', 'Zimbabwe\'s army seizes power, Mugabe confined but "safe"', 'U.S. senator urges DOJ to reject any White House push in merger probes']


[' "saf',
 ' alex',
 ' any ',
 ' army',
 ' bash',
 ' bios',
 ' bipa',
 ' but ',
 ' cele',
 ' comp',
 ' conf',
 ' cons',
 ' deve',
 ' doj ',
 ' fete',
 ' heal',
 ' hold',
 ' hous',
 ' in ',
 ' jong',
 ' kim ',
 ' kore',
 ' merg',
 ' muga',
 ' nort',
 ' nucl',
 ' offi',
 ' on ',
 ' plan',
 ' powe',
 ' prob',
 ' push',
 ' reje',
 ' repu',
 ' scie',
 ' seiz',
 ' sena',
 ' stra',
 ' to ',
 ' u.s.',
 ' un ',
 ' urge',
 ' whit',
 ' zimb',
 '"safe',
 "abwe'",
 'afe" ',
 'alexa',
 'althc',
 'ander',
 'army ',
 'artis',
 'ategy',
 'ation',
 'ator ',
 'babwe',
 'bash ',
 'biose',
 'bipar',
 'blica',
 'brati',
 "bwe's",
 'care ',
 'celeb',
 'cial ',
 'cient',
 'clear',
 'compr',
 'confi',
 'consu',
 'curit',
 'devel',
 "ea's ",
 'ealth',
 'ebrat',
 'ecuri',
 'egy: ',
 'ehens',
 'eizes',
 'eject',
 'elebr',
 'elopi',
 'enato',
 'ensiv',
 'entis',
 'epubl',
 'erger',
 'etes ',
 'evelo',
 'exand',
 'fetes',
 'ffici',
 'ficia',
 'fined',
 'gabe ',
 'hcare',
 'healt',
 'hensi',
 'hite ',
 'holds',
 'ho

In [14]:
# create a pickle file for the vocabulary
def pikle_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
def dill_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        dill.dump(obj, output)        

In [ ]:
## pickle the Vocabulary
pikle_object(vocab, 'parsed_data/vocab.pkl')

#Save the unknown sequence for analysis
np.save('parsed_data/GloVe_Unknown_50.npy', glove_dd['unk'])

In [19]:
#pdio.save_pandas(r"parsed_data/df_alldata_embed.pdy", all_data) ## does not work for objects
#all_data.to_msgpack('parsed_data/df_alldata_embed.msg') ## error

#store = pd.HDFStore('parsed_data/pdata.h5')
#store['df_alldata_embed'] = all_data   ## cant save complex objects

#all_data1.to_hdf('parsed_data/pdata.h5' , 'df_alldata_embed', format = 'table', mode = 'w' ) #kernel restarts

#all_data.to_parquet('parsed_data/df_alldata_embed.parquet') #embed column cannot be parquet

#all_data.to_json('parsed_data/df_alldata_embed.json')  #kernel restarts
#all_data.to_feather('parsed_data/df_alldata_embed.feather')  #cant use on embeddings
#all_data.to_csv('parsed_data/df_alldata_embed.csv', chunksize =5000 ) #breaks embeddings


#### Thanks Mike Mckerns for creating dill, the hammer of pickle!!!!

dill_object(all_data, 'parsed_data/df_alldata_embed.pkl')

In [20]:
!ls ./parsed_data -lah

total 4.9G
drwxrwxr-x 2 w266project w266project 4.0K Nov 17 05:16 .
drwxrwxr-x 9 w266project w266project 4.0K Nov 17 05:18 ..
-rw-rw-r-- 1 w266project w266project  528 Nov 17 02:04 GloVe_Unknown_50.npy
-rw-rw-r-- 1 w266project w266project  95M Nov 12 06:53 df_alldata1.pkl
-rw-rw-r-- 1 w266project w266project 368M Nov 17 02:01 df_alldata2.pkl
-rw-rw-r-- 1 w266project w266project 4.5G Nov 17 05:19 df_alldata_embed.pkl
-rw-rw-r-- 1 w266project w266project 2.3K Nov 17 04:28 pdata.h5
-rw-rw-r-- 1 w266project w266project  34M Nov 17 02:04 vocab.pkl
